In [14]:
import psycopg2 as ps
import ipyleaflet as ipyl
import ipywidgets as ipyw
import pandas as pd
import json


In [34]:
conn_string = "dbname='gis' user='postgres' password='123456' host='localhost'"
conn = ps.connect(conn_string)
cur = conn.cursor()
query = """SELECT 
               * 
           FROM 
               planet_osm_point p 
           INNER JOIN 
               planet_osm_nodes n ON n.id=p.osm_id
"""
query_json = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT osm_id, access) As l
      )) As properties
   FROM planet_osm_line As lg   ) As f )  As fc;
"""

cur.execute(query_json)
d = cur.fetchone()

df = pd.read_sql(query, conn)
conn.close()

In [104]:
map = ipyl.Map(center=[52.52, 13.45], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

layer = ipyl.GeoJSON(data=d[0], hover_style={'fillColor': 'red'})

def hover_handler(event=None, id=None, properties=None):
    props = [properties['osm_id'], properties['access']]
    prop_list = [str(x) for x in props]
    label.value = '\n'.join(prop_list)
    
layer.on_hover(hover_handler)
map.add_layer(layer)


ipyw.VBox([map, label])



VBox(children=(Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[52.52, 13.45], layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2516230490082, 52.4023272722824], [13.2515421108011, 52.402450862755]]}, 'properties': {'osm_id': 233662253, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2518337039423, 52.4023833950056], [13.2517445910661, 52.4025068757065]]}, 'properties': {'osm_id': 233662252, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2730883825538, 52.4971916004279], [13.272474204394, 52.4969777094186], [13.2719805801454, 52.4967441836562], [13.2715026764142, 52.4964839677744], [13.2710189336337, 52.4961615123299], [13.2704865919964, 52.4957594788325], [13.2699179584215, 52.4953250641349], [13.2693601046301, 52.4949054120706], [13.2688097068555, 52.4944832948241], [13.268265776951, 52.4940659865405]]}, 'properties': {'osm_id': 320870812, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2404351609652, 52.5357648652968], [13.2373805296731, 52.5348662770895], [13.2368320183606, 52.5337811711181], [13.2417262196915, 52.5319047673847], [13.2460606807689, 52.5331527066562], [13.2490385061043, 52.5320009993712]]}, 'properties': {'osm_id': -3764521, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2404351609652, 52.5357648652968], [13.2373805296731, 52.5348662770895], [13.2368320183606, 52.5337811711181], [13.2417262196915, 52.5319047673847], [13.2460606807689, 52.5331527066562], [13.2490385061043, 52.5320009993712]]}, 'properties': {'osm_id': 11499908, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2486699273432, 52.5307845044389], [13.2514899186831, 52.528125268309], [13.25768515004, 52.5280031778522], [13.2636050477624, 52.5291928075588], [13.2693392637155, 52.529531470138], [13.2755722243144, 52.5289614749784], [13.2790767319008, 52.5276116007534], [13.2809923892441, 52.5265847941485]]}, 'properties': {'osm_id': 43163137, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2486699273432, 52.5307845044389], [13.2514899186831, 52.528125268309], [13.25768515004, 52.5280031778522], [13.2636050477624, 52.5291928075588], [13.2693392637155, 52.529531470138], [13.2755722243144, 52.5289614749784], [13.2790767319008, 52.5276116007534], [13.2809923892441, 52.5265847941485]]}, 'properties': {'osm_id': -3749523, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.2480260149475, 52.5313357866735], [13.2484223516509, 52.5312770957735], [13.2512431514745, 52.529450206659], [13.2553477336707, 52.5293552806696], [13.2590749336161, 52.5292537965265], [13.2636050477624, 52.5291928075588], [13.2676395613664, 52.5285476636946], [13.270959644825, 52.528633191571], [13.2741852255157, 52.5287201948372], [13.2774271555445, 52.5287838623192], [13.280933549593, 52.5288638700798], [13.2827353005584, 52.5315051919971], [13.2851487144007, 52.5318638919512], [13.2880076926239, 52.5322926995841], [13.2903420547213, 52.5326328136839]]}, 'properties': {'osm_id': 11500346, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.248908070725, 52.5318727993016], [13.2114984493701, 52.5805116058384]]}, 'properties': {'osm_id': 381739512, 'access': None}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[13.248908070725, 52.5318727993016], [13.2114984493701, 52.5805116058384]]}, 'properties': {'osm_id': -5681704, 'access': None}}, {'type': 'Feature'

In [101]:
x=1
y=2
X=[x,y]
x=[str(x) for x in X]
print (type('\n'.join(x)))



<class 'str'>
